In [1]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import IPython as ip

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import yacht_main as yacht
from yacht_test import create_train_set

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
torch.cuda.is_available()

True

In [2]:
INPUT_SIZE = 44
OUTPUT_SIZE = 43

class DQN(nn.Module):

    def __init__(self, input_size, output_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_size, 300)
        self.fc2 = nn.Linear(300, 300)
        self.fc3 = nn.Linear(300, 50)
        #self.fc4 = nn.Linear(100, 100)
        #self.fc5 = nn.Linear(100, 100)
        self.fc6 = nn.Linear(50, output_size)
        
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        #x = F.relu(self.fc4(x))
        #x = F.relu(self.fc5(x))

        x = self.fc6(x)

        return x

    
policy_net = DQN(INPUT_SIZE,OUTPUT_SIZE).to(device)

In [3]:
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 500

steps_done = 0

def select_action(state, avail):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max (1)은 각 행의 가장 큰 열 값을 반환합니다.
            # 최대 결과의 두번째 열은 최대 요소의 주소값이므로,
            # 기대 보상이 더 큰 행동을 선택할 수 있습니다.
            return (policy_net(state) * avail).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(OUTPUT_SIZE)]], device=device, dtype=torch.long)

In [4]:
yacht.roll_dice(0)

[5, 5, 5, 4, 3]

In [5]:
handled_weight = [0.2, 0.2, 0.2, 0.2, 0.1, 0.1]
random.choices(population= range(6), weights = handled_weight)

[5]

In [6]:
# straight_3, straight_4, straight_5, "fulh", "tri", "four", "yacht"
handled_weight = [0.4, 0.2, 0.2, 0.1, 0.1]
category = random.choices(population=range(5), weights=handled_weight)
# straight_3
if category == 0:
    bot = randint(1, 4)
    seq = random.shuffle(range(5))
    dice_status[seq[0]] = bot
    dice_status[seq[1]] = bot + 1
    dice_status[seq[2]] = bot + 2
    remain_value = (range(6) + 1).remove(bot-1)
    remain_value.remove(bot + 3)
    dice_status[seq[3]] = random.choice(remain_value)
    dice_status[seq[4]] = random.choice(remain_value)

In [7]:
dice_status = [random.randint(1, 6), random.randint(1, 6), random.randint(1, 6), random.randint(1, 6), random.randint(1, 6)]

In [8]:
bot = random.randint(1, 4)
seq = random.sample(list(range(5)), 5)
for i in range(3):
    dice_status[seq[i]] = bot + i
remain_value = list(range(1, 7))
if bot - 1 > 0:
    remain_value.remove(bot -1)
if bot + 3 < 7:
    remain_value.remove(bot + 3)
dice_status[seq[3]] = random.choice(remain_value)
dice_status[seq[4]] = random.choice(remain_value)

In [9]:
seq = random.sample(list(range(5)), 5)
bot = random.choice([1,3])
for i in range(4):
    dice_status[seq[i]] = bot + i
remain_value = list(range(1,7))
if bot == 1:
    remain_value.remove(5)
else:
    remain_value.remove(2)
dice_status[seq[4]] = random.choice(remain_value)
dice_status

[2, 3, 4, 2, 1]

In [10]:
seq = random.sample(list(range(5)), 5)
bot = random.randint(1,2)
for i in range(5):
    dice_status[seq[i]] = bot + i
dice_status

[6, 2, 5, 4, 3]

In [11]:
seq = random.sample(list(range(5)), 5)
fh_values = random.sample(list(range(1, 7)), k=3)
for i in range(4):
    dice_status[seq[i]] = fh_values[i//2]
dice_status[seq[4]] = fh_values[2]
dice_status

[3, 5, 1, 3, 5]

In [12]:
seq = random.sample(list(range(5)), 5)
fh_values = random.sample(list(range(1, 7)), k=2)
for i in range(4):
    dice_status[seq[i]] = fh_values[i//2]
dice_status[seq[4]] = fh_values[1]
dice_status

[1, 5, 1, 1, 5]

In [13]:
seq = random.sample(list(range(5)), 5)
tri_values = random.sample(list(range(1, 7)), k=3)
for i in range(3):
    dice_status[seq[i]] = tri_values[0]
dice_status[seq[3]] = tri_values[1]
dice_status[seq[4]] = tri_values[2]
dice_status

[6, 2, 3, 6, 6]

In [14]:
seq = random.sample(list(range(5)), 5)
four_values = random.sample(list(range(1, 7)), k=2)
for i in range(5):
    dice_status[seq[i]] = four_values[i//4]
dice_status

[1, 1, 1, 2, 1]

In [15]:
value = random.randint(1, 6)
for i in range(5):
    dice_status[i] = value
dice_status

[2, 2, 2, 2, 2]

In [16]:
def handled_roll():
        # straight_3, straight_4, straight_5, 212, fh, 311, 41, yacht
        handled_weight = [5, 3, 2, 3, 2, 4, 2, 1]
        category = random.choices(population=range(len(handled_weight)), weights=handled_weight)[0]
        print(category)
        # straight_3
        if category == 0:
            bot = random.randint(1, 4)
            seq = random.sample(list(range(5)), 5)
            for i in range(3):
                dice_status[seq[i]] = bot + i
            remain_value = list(range(1, 7))
            if bot - 1 > 0:
                remain_value.remove(bot -1)
            if bot + 3 < 7:
                remain_value.remove(bot + 3)
            dice_status[seq[3]] = random.choice(remain_value)
            dice_status[seq[4]] = random.choice(remain_value)
        # straight_4
        elif category == 1:
            seq = random.sample(list(range(5)), 5)
            bot = random.choice([1,3])
            for i in range(4):
                dice_status[seq[i]] = bot + i
            remain_value = list(range(1,7))
            if bot == 1:
                remain_value.remove(5)
            else:
                remain_value.remove(2)
            dice_status[seq[4]] = random.choice(remain_value)
        # straight_5
        elif category == 2:
            seq = random.sample(list(range(5)), 5)
            bot = random.randint(1,2)
            for i in range(5):
                dice_status[seq[i]] = bot + i
        # fh_212
        elif category == 3:
            seq = random.sample(list(range(5)), 5)
            fh_values = random.sample(list(range(1, 7)), k=3)
            for i in range(4):
                dice_status[seq[i]] = fh_values[i//2]
            dice_status[seq[4]] = fh_values[2]
        # fh
        elif category == 4:
            seq = random.sample(list(range(5)), 5)
            fh_values = random.sample(list(range(1, 7)), k=2)
            for i in range(4):
                dice_status[seq[i]] = fh_values[i//2]
            dice_status[seq[4]] = fh_values[1]
        # 311
        elif category == 5:
            seq = random.sample(list(range(5)), 5)
            tri_values = random.sample(list(range(1, 7)), k=3)
            for i in range(3):
                dice_status[seq[i]] = tri_values[0]
            dice_status[seq[3]] = tri_values[1]
            dice_status[seq[4]] = tri_values[2]
        # 41
        elif category == 6:
            seq = random.sample(list(range(5)), 5)
            four_values = random.sample(list(range(1, 7)), k=2)
            for i in range(5):
                dice_status[seq[i]] = four_values[i//4]
        # yacht
        elif category == 7:
            value = random.randint(1, 6)
            for i in range(5):
                dice_status[i] = value
        return dice_status

In [17]:
handled_roll()

6


[2, 2, 2, 2, 4]

In [18]:
yacht.handled_roll()

NameError: name 'random' is not defined

In [ ]:
import yacht_main
yacht_main.handled_roll()